<a href="https://colab.research.google.com/github/Gyeong-Hyeon/TRANSLATE-YOUR-PHOTO/blob/main/translater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* https://manuscriptlink-society-file.s3-ap-northeast-1.amazonaws.com/kips/conference/2020fall/presentation/KIPS_C2020B0187.pdf

* https://www.tensorflow.org/tutorials/text/nmt_with_attention

* https://github.com/Kyubyong/cjk_trans

번역 모델 선정
* https://tech.kakaoenterprise.com/48

Tokenizer:

* English: https://github.com/alvations/sacremoses

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Prepare dataset

In [ ]:
import pandas as pd

wr = pd.read_excel('/content/drive/MyDrive/OCR/dataset/koen/4_문어체_한국문화_200226.xlsx')
sp = pd.read_excel('/content/drive/MyDrive/OCR/dataset/koen/1_구어체(1)_200226.xlsx')
sp2 = pd.read_excel('/content/drive/MyDrive/OCR/dataset/koen/1_구어체(2)_200226.xlsx')
n = pd.read_excel('/content/drive/MyDrive/OCR/dataset/koen/3_문어체_뉴스(1)_200226.xlsx')
n2 = pd.read_excel('/content/drive/MyDrive/OCR/dataset/koen/3_문어체_뉴스(2)_200226.xlsx')
n3 = pd.read_excel('/content/drive/MyDrive/OCR/dataset/koen/3_문어체_뉴스(3)_200226.xlsx')
n4 = pd.read_excel('/content/drive/MyDrive/OCR/dataset/koen/3_문어체_뉴스(4)_200226.xlsx')

wr.head(1)

In [ ]:
sp.head(1)

In [ ]:
n.head(1)

In [ ]:
wr.drop(columns='키워드', inplace=True)
sp = sp.rename({'SID':'ID'}, axis='columns')
sp2 = sp2.rename({'SID':'ID'}, axis='columns')
n.drop(columns=['날짜','자동분류1','자동분류2','자동분류3','URL','언론사'], inplace=True)
n2.drop(columns=['날짜','자동분류1','자동분류2','자동분류3','URL','언론사'], inplace=True)
n3.drop(columns=['날짜','자동분류1','자동분류2','자동분류3','URL','언론사'], inplace=True)
n4.drop(columns=['날짜','자동분류1','자동분류2','자동분류3','URL','언론사'], inplace=True)

df = pd.concat([wr,sp,sp2,n,n2,n3,n4])
print(len(df),len(wr)+len(sp)+len(sp2)+len(n)+len(n2)+len(n3)+len(n4))

In [ ]:
df.reset_index(drop=True, inplace=True)
df

In [ ]:
df['번역문'].to_csv('en.txt',index=False)

##Preprocessing

In [ ]:
!pip install -U sacremoses
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [ ]:
%cd /content/drive/MyDrive/OCR/dataset

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab().morphs
sentence=df[k][0]
morph_sentence= []
count = 0
for token_mecab in mecab(sentence):
    token_mecab_save = token_mecab
    if count > 0:
        token_mecab_save = "##" + token_mecab_save  # 앞에 ##를 부친다
        morph_sentence.append(token_mecab_save)
    else:
        morph_sentence.append(token_mecab_save)
        count += 1
# 문장단위 저장
print('mecab check :', ' '.join(morph_sentence))

In [ ]:
import unicodedata
import string
import re
from konlpy.tag import Mecab
from sacremoses import MosesTokenizer

class preprocessing:
    
    def __init__(self):
        punct = '"“”#$%&\'()*+,-/:;<=>@[\\]^_`{|}~'
        self.table = str.maketrans({key: None for key in punct})
    
    @staticmethod
    def unicodeToAscii(s):
        print('unicode')
        s= ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )
        print(s)
        return s

    # Lowercase, trim, and remove non-letter characters
    @staticmethod
    def normalizeString(s):
        s = preprocessing.unicodeToAscii(s.lower().strip())
        s = re.sub(r"([.!?])", r" \1", s)
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
        print('normal',s)
        return s
    
    def extract_stem(self, df, lang= 'kr', generation=True):
        sentences = []
        if lang == 'kr':
          for s in df:
            sentence = unicodedata.normalize('NFKC', s.strip()).translate(self.table)
            if generation:
              punct = '"“”#$%&\'()*+,-/:;<=>@[\\]^_`{|}~'
              sentence = unicodedata.normalize('NFKC', s.strip()).translate(str.maketrans({key: None for key in punct}))
              token_mecab_save = mecab_tokenizer(sentence)
              split = sentence.split()
              spaced = []
              for i,tk in enumerate(token_mecab_save):
                token_mecab_save[i] = tk.strip()
                
              for w in split:
                  if w == token_mecab_save[0]:                  
                      spaced.append(token_mecab_save[0])
                      del token_mecab_save[0]
                  else:
                      spaced.append(token_mecab_save[0])
                      temp = token_mecab_save[0]
                      del token_mecab_save[0]
                      while token_mecab_save:
                          if w != temp:
                              spaced.append("##" + token_mecab_save[0]) # 앞에 ##를 붙인다
                              temp = temp+token_mecab_save[0]
                              del token_mecab_save[0]
                          else:
                              break
            else:
              spaced= mecab_tokenizer(sentence)
            sentences.append(spaced)
        elif lang == 'en':
          for s in df:
            spaced = mt.tokenize(self.normalizeString(s))
            sentences.append(spaced)
        # else:
        #     print('pick English or Korean')
        return sentences

In [ ]:
import time
k='원문'
e='번역문'

data = df.copy()

mecab_tokenizer=Mecab().morphs
mt = MosesTokenizer()

en_ko_pre = preprocessing()

start = time.time()
X = en_ko_pre.extract_stem(df[e],lang='en')
Y = en_ko_pre.extract_stem(data[k],lang='kr')
  
with open(f'after_mecab_kr.txt', 'w', encoding='utf-8') as f:
    for line in Y:
        f.write(' '.join(line)+'\n')

with open(f'after_mecab_en.txt', 'w', encoding='utf-8') as f:
    for line in X:
        f.write(' '.join(line)+'\n')
f.close()

print('Total preprocessing time:', time.time()-start)
print(X)
print(Y)

##Tokenizing

In [ ]:
import pandas as pd

df = pd.read_table('/content/drive/MyDrive/OCR/dataset/after_mecab_en.txt', sep='\n',header=None,names=['input'])
Y = pd.read_table('/content/drive/MyDrive/OCR/dataset/after_mecab_kr.txt', sep='\n',header=None,names=['target'])


df['target'] = Y
df

,input,target
0,gangneung maehwajeon one of the twelve madang ...,강릉 기생 매화 ##가 등장 ##하 ##는 판소리 열 ##두 ##마당 ##의 하나 ...
1,the purpose of the establishment was to hold v...,다양 ##한 미술 ##관련 전시회 ##의 개최 각종 교육 ##프로그램 ##과 새로운...
2,there are two main types of soy sauce home bre...,간장 ##은 가정 ##에서 담구 ##던 재래식 간장 ##과 공장 ##에서 양조 ##...
3,he occupies a significant position in korean h...,그 ##는 한국 ##에 발 ##을 내딛 ##은 최조 ##의 가톨릭 사제 ##로 한국...
4,it is unclear regarding the foundation of deok...,덕사 ##의 창건 ##은 신라 말 고려 초 ##라고 전해 ##지나 문헌 ##이 없 ...
...,...,...
1302028,the proposed reduction of the service period w...,복무 ##기간 단축 ##안 ##은 10 ##월 전 ##역자 ##부터 2 ##주 단위...
1302029,in fact the vice chief of the seoul eastern di...,실제로 이번 인사 ##에서 인천 ##지검 특수 ##부장 서울 ##중앙 ##지검 증권...
1302030,on the th at a meeting of experts on how to tr...,29 ##일 서울 서초구 한국 ##산업 ##기술 ##진흥 ##협회 중 ##회의실 #...
1302031,the gwangju office of education said there was...,광주시 ##교육청 ##은 지난 1 ##일 ##과 2 ##일 한 ##유총 광주 ##지...


In [ ]:
tk = pd.concat([df['input'],Y['target']])
tk.to_csv('after_mecab.txt', sep = '\t', index=False)

In [ ]:
!pip install sentencepiece
!pip install transformers

In [ ]:
%cd /content/drive/MyDrive/OCR/model

In [ ]:
import os
import sentencepiece as spm

vocab_size = 64000

model_type = 'unigram'
character_coverage  = 1.0  # 전체를 cover 하기 위해, default=0.9995
user_defined_symbols = '[SEP],[MASK],[BOS],[EOS],[UNK0],[UNK1],[UNK2],[UNK3],[UNK4],[UNK5],[UNK6],[UNK7],[UNK8],[UNK9],[unused0],[unused1],[unused2],[unused3],[unused4],[unused5],[unused6],[unused7],[unused8],[unused9],[unused10],[unused11],[unused12],[unused13],[unused14],[unused15],[unused16],[unused17],[unused18],[unused19],[unused20],[unused21],[unused22],[unused23],[unused24],[unused25],[unused26],[unused27],[unused28],[unused29],[unused30],[unused31],[unused32],[unused33],[unused34],[unused35],[unused36],[unused37],[unused38],[unused39],[unused40],[unused41],[unused42],[unused43],[unused44],[unused45],[unused46],[unused47],[unused48],[unused49],[unused50],[unused51],[unused52],[unused53],[unused54],[unused55],[unused56],[unused57],[unused58],[unused59],[unused60],[unused61],[unused62],[unused63],[unused64],[unused65],[unused66],[unused67],[unused68],[unused69],[unused70],[unused71],[unused72],[unused73],[unused74],[unused75],[unused76],[unused77],[unused78],[unused79],[unused80],[unused81],[unused82],[unused83],[unused84],[unused85],[unused86],[unused87],[unused88],[unused89],[unused90],[unused91],[unused92],[unused93],[unused94],[unused95],[unused96],[unused97],[unused98],[unused99]'
# unused_token_num = 180
# unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
# user_defined_symbols = user_defined_symbols + unused_list

print(user_defined_symbols)

In [ ]:
max_len=0
for i in df['input']:
  if len(i) > max_len:
    max_len = len(i)

for i in df['target']:
  if len(i) > max_len:
    max_len = len(i) 

print(max_len)

In [ ]:
input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --model_type=%s --character_coverage=%s --max_sentence_length=786 --pad_id=0 --pad_piece=[PAD] --bos_id=1 --bos_piece=[CLS] --eos_id=3  --eos_piece=[SEP] --unk_id=2 --unk_piece=[UNK] --user_defined_symbols=%s'

cmd = input_argument%('/content/drive/MyDrive/OCR/dataset/after_mecab.txt', 'spm_tokenizer', vocab_size, model_type, character_coverage, user_defined_symbols)

spm.SentencePieceTrainer.Train(cmd)

In [ ]:
print(cmd)

In [ ]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.Load('/content/drive/MyDrive/OCR/model/spm/spm_tokenizer.model')

mecab_tokenizer=Mecab().morphs
mt = MosesTokenizer()

en_ko_pre = preprocessing()

#sentence = 'gangneung maehwajeon one of the twelve madang of pansori . '
sentence = en_ko_pre.extract_stem(["I'm in love with you."],lang='en')
for line in sentence:
    sentence = ' '.join(line)+'\n'

print(sentence)
tokens = sp.encode_as_pieces(sentence)
ids = sp.encode_as_ids(sentence)

print(ids)
print(type(ids))
print(sp.piece_to_id(tokens))
print(tokens)

# tokens = sp.decode_pieces(tokens)
# ids = sp.decode_ids(ids)

# print(ids)
# print(tokens)

max_len = 13
n_to_pad = max_len - len(tokens)


tokens = ['[CLS]', ] + sp.encode_as_pieces(sentence) + ['[PAD]', ]*n_to_pad + ['[SEP]', ]

print(tokens)
print(sp.piece_to_id(tokens))

unicode
i'm in love with you.
normal i m in love with you .
i m in love with you .

[155, 459, 134, 1134, 164, 183, 119]
<class 'list'>
[155, 459, 134, 1134, 164, 183, 119]
['▁i', '▁m', '▁in', '▁love', '▁with', '▁you', '▁.']
['[CLS]', '▁i', '▁m', '▁in', '▁love', '▁with', '▁you', '▁.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[SEP]']
[1, 155, 459, 134, 1134, 164, 183, 119, 0, 0, 0, 0, 0, 0, 3]


In [ ]:
%cd /content/drive/MyDrive/OCR/dataset

In [ ]:
#CLS=1, SEP=3
#데이터 단위가 클 경우 tf.utils.Sequence 사용 - https://jins-sw.tistory.com/6
from tqdm import tqdm
import sentencepiece as spm
import numpy as np

sp = spm.SentencePieceProcessor()
sp.Load('/content/drive/MyDrive/OCR/model/spm_tokenizer.model')

def tokenizing(df, max_len=0):
    ids=[]
    #tokens=[]
    # for s in tqdm(df, desc='tokenizing'):
    #     token = sp.encode_as_pieces(s)
    #     tokens.append(token) #append token

    # if max_len == 0: #check max length of token
    #     max_len = max(len(token) for token in tokens)
    
    #i=0
    for s in tqdm(df, desc='padding'):      
        # if len(tokens) >= max_len: #Truncate if tokens are longer than max_len
        #     result.append(tokens[:max_len])
        # else:
        token = sp.encode_as_pieces(s)
        n_to_pad = max_len - len(token) 
        token = ['[CLS]', ] + token + ['[PAD]', ]*n_to_pad + ['[SEP]', ]
        ids.append(sp.piece_to_id(token)) #append id
        #i+=1
    
    ids = np.array(ids)
    
    return ids

id_input = tokenizing(df['input'], max_len=145)
id_output = tokenizing(df['target'], max_len=220)

In [ ]:
#df[['id_input','id_output','token_input','token_output']].to_pickle('/content/drive/MyDrive/OCR/dataset/tokenized')

In [ ]:
#df[['id_input','id_output','token_input','token_output']].to_csv('/content/drive/MyDrive/OCR/dataset/tokenized.csv',index=False) #Recommend saving df into pickle

In [ ]:
import pickle

filename ='/content/drive/MyDrive/OCR/dataset/id_input.pkl'
pickle.dump(id_input, open(filename, 'wb'))

filename = '/content/drive/MyDrive/OCR/dataset/id_output.pkl'
pickle.dump(id_output, open(filename, 'wb'))

In [ ]:
def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format(MODEL_PREFIX))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

##Make dataset for fine-tuning

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
%cd /content/drive/MyDrive/OCR

/content/drive/MyDrive/OCR


In [ ]:
# import pandas as pd

# #df = pd.read_csv('/content/drive/MyDrive/OCR/dataset/df_pp.csv', converters={'id_input': eval,'token_input': eval,'token_output': eval,'id_output': eval})
# df = pd.read_pickle('/content/drive/MyDrive/OCR/dataset/tokenized')

# df

In [ ]:
import pickle

with open('dataset/id_input.pkl', 'rb') as f:
    X = pickle.load(f)

with open('dataset/id_output.pkl', 'rb') as f:
    y = pickle.load(f)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.008, random_state=10, shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1291616, 147), (10417, 147), (1291616, 222), (10417, 222))

In [ ]:
import torch
from torch.utils.data import DataLoader,TensorDataset

X_train_ts = torch.from_numpy(X_train)
X_test_ts = torch.from_numpy(X_test)
y_train_ts = torch.from_numpy(y_train)
y_test_ts = torch.from_numpy(y_test)

train_tensor = TensorDataset(X_train_ts, y_train_ts)
test_tensor = TensorDataset(X_test_ts, y_test_ts)

train_loader = DataLoader(train_tensor, batch_size=128, shuffle=True)

In [ ]:
from transformers import BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderModel, BertGenerationConfig

tokenizer = Berttoken
encoder = BertGenerationEncoder.from_pretrained("monologg/kobert")
decoder = BertGenerationDecoder.from_pretrained("bert-large-uncased", add_cross_attention=True, is_decoder=True)
model = EncoderDecoderModel(encoder=encoder, decoder=decoder)
#optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
!pip install pytorch_pretrained_bert

In [ ]:
from pytorch_pretrained_bert import BertAdam
# Prepare optimizer
param_optimizer = list(model.named_parameters())

# hack to remove pooler, which is not used
# thus it produce None grad that break apex
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
optim = BertAdam(optimizer_grouped_parameters,
                             lr=5e-5,
                             warmup=0.1,
                             t_total=300)

In [ ]:
import time
from tqdm import tqdm

model.to('cuda')
model.train()

losses = []
for epoch in range(50):
    start = time.time()
    for batch_X, batch_y in tqdm(train_loader, desc=f'Epoch{epoch}'):
        for seq_X, seq_y in zip(batch_X, batch_y):
            optim.zero_grad()
            seq_X = batch_X.to('cuda')
            seq_y = batch_y.to('cuda')
            loss = model(input_ids=batch_X, decoder_input_ids=batch_y, labels=batch_y).loss
            eveloss += loss.mean().item()
            loss.backward()
            optim.step()
    losses.append(eve_loss)
    print("Epoch",epoch,">> LOSS : ",eveloss," TRAINING TIME : ", time.time()-start)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(losses)
plt.show()

##Another tokenizer

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets

In [ ]:
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBartTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
from torch.utils.data import random_split

ImportError: ignored

In [ ]:
data = []
for src, tgt in zip(df['input'],df['target']):
      data.append(
          {
              "translation": {
                  "en": src.strip(),
                  "ko": tgt.strip()
              }
          }
      )
print(f'total size of data is {len(data)}')

total size of data is 1302033


In [ ]:
max_input_length = 147
max_target_length = 222
source_lang = "en_XX"
target_lang = "ko"

def data_collator(features:list):
  labels = [f["translation"]["ko"] for f in features]
  inputs = [f["translation"]["en"] for f in features]

  batch = tokenizer.prepare_seq2seq_batch(src_texts=inputs, src_lang=source_lang, tgt_lang=target_lang, tgt_texts=labels, max_length=max_input_length, max_target_length=max_target_length)

  for k in batch:
    batch[k] = torch.tensor(batch[k])

  return batch

In [ ]:
# initiating model, tokenizer
model = MBartForConditionalGeneration.from_pretrained("hyunwoongko/kobart")
tokenizer = MBartTokenizer.from_pretrained("hyunwoongko/kobart")

# defining training related arguments
args = Seq2SeqTrainingArguments(output_dir="/content/drive/MyDrive/OCR/model/translator",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=64,
                        per_device_eval_batch_size=64,
                        learning_rate=5e-5,
                        num_train_epochs=2,
                        logging_dir="/logs")

NameError: ignored

In [ ]:
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                data_collator=data_collator, 
                train_dataset=[X_train, y_train], 
                eval_dataset=eval_dataset)

##Git 가져오기

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.008, random_state=10, shuffle=True)
train.shape, test.shape

((1291616, 2), (10417, 2))

In [ ]:
%cd /content/drive/MyDrive/OCR

/content/drive/MyDrive/OCR


In [ ]:
 !pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

In [ ]:
!git clone https://github.com/seujung/KoBART-translation.git

Cloning into 'KoBART-translation'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 46 (delta 18), reused 32 (delta 8), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [ ]:
%cd KoBART-translation

/content/drive/My Drive/OCR/KoBART-translation


In [ ]:
train.to_csv('data/train.tsv', sep='\t')
test.to_csv('data/test.tsv', sep='\t')

In [ ]:
!./prepare.sh
!pip install -r requirements.txt
!python train.py  --gradient_clip_val 1.0 --max_epochs 50 --default_root_dir logs  --gpus 1 --batch_size 4

##Following transformer paper

In [ ]:
!pip install torchtext==0.6.0

In [ ]:
import spacy

spacy_en = spacy.load('en') # 영어 토큰화(tokenization)

tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]